In [ ]:
import numpy as np
import keyboard
from IPython.display import clear_output
from src.constants import constants as cts
from src.model.MDP_2048 import MDP_2048

def get_action(last):
    curr = np.zeros(4, dtype=int)
    if keyboard.is_pressed("right"):
        curr[0] = 1
    if keyboard.is_pressed("left"):
        curr[1] = 1
    if keyboard.is_pressed("up"):
        curr[2] = 1
    if keyboard.is_pressed("down"):
        curr[3] = 1

    if curr.sum() != 1 or np.equal(last, curr).all():
        return None, curr

    if curr[0] == 1:
        return cts.MOVE_RIGHT, curr
    if curr[1] == 1:
        return cts.MOVE_LEFT, curr
    if curr[2] == 1:
        return cts.MOVE_UP, curr
    if curr[3] == 1:
        return cts.MOVE_DOWN, curr

last_key = np.zeros(4, dtype=int)
reward = 0
model = MDP_2048()
model.initialize_state()
print("SCORE:", reward)
print("\n\n\n")
model.print_state()

while not keyboard.is_pressed("esc"):

    action, last_key = get_action(last_key)

    if action is not None:
        model.transition_function(action)
        reward = model.reward_function()

        clear_output(wait=True)
        print("SCORE:", reward)
        print(f"\n{action}\n")
        model.print_state()

print("\nGAME ENDED, SCORE =", reward)

In [ ]:
def __rotate_before_compact(state, action):
    if action == cts.MOVE_RIGHT:
        return np.flip(state, axis=1)
    elif action == cts.MOVE_UP:
        return state.T
    elif action == cts.MOVE_DOWN:
        return np.flip(state.T, axis=1)
    else:
        return state

def __compact_tiles(state):
    def compact_row(row):
        length = len(row)
        row = row[np.where(row > 0)]
        for i in range(len(row) - 1):
            if row[i] == row[i + 1] & row[i] != 0:
                row[i] += 1
                row[i + 1] = 0
        row = row[np.where(row > 0)]
        row = np.pad(row, (0, length - len(row)), 'constant', constant_values=0)
        return row

    return np.apply_along_axis(compact_row, 1, state)

def __rotate_after_compact(state, action):
    if action == cts.MOVE_RIGHT:
        return np.flip(state, axis=1)
    elif action == cts.MOVE_UP:
        return state.T
    elif action == cts.MOVE_DOWN:
        return np.flip(state, axis=1).T
    else:
        return state


s = np.array([[1,2,1,0],[1,2,0,0],[3,0,0,0],[3,0,0,1]])
action = cts.MOVE_RIGHT

print(s, "init")
s = __rotate_before_compact(s, action)
print(s, "after first rotate")
s = __compact_tiles(s)
print(s, "after compact")
s = __rotate_after_compact(s, action)
print(s, "after second rotate")

In [ ]:
def compact_row(row):
    length = len(row)
    row = row[np.where(row > 0)]
    print(row)
    for i in range(len(row) - 1):
        if row[i] == row[i + 1] and row[i] != 0:
            row[i] += 1
            row[i + 1] = 0
    print(row)
    row = row[np.where(row > 0)]
    print(row)
    row = np.pad(row, (0, length - len(row)), 'constant', constant_values=0)
    print(row)
    return row

r = np.array([1, 3, 4, 1])
print(r)
r = compact_row(r)
print(r)

In [15]:
import numpy as np

def __compact_tiles(state):
    def compact_row(row, reward):
        length = len(row)
        row = row[np.where(row > 0)]
        for i in range(len(row) - 1):
            if row[i] == row[i + 1] and row[i] != 0:
                row[i] += 1
                row[i + 1] = 0
                reward[0] += 2 ** row[i]
        row = row[np.where(row > 0)]
        row = np.pad(row, (0, length - len(row)), 'constant', constant_values=0)
        return row

    reward = [0]
    state_out = np.apply_along_axis(compact_row, 1, state, reward)

    return state_out, reward[0]

m = np.array([
    [1,2,3,4],
    [5,6,2,8],
    [9,2,11,2],
    [13,14,2,2]
])

print(m)

__compact_tiles(m)

[[ 1  2  3  4]
 [ 5  6  2  8]
 [ 9  2 11  2]
 [13 14  2  2]]


(array([[ 1,  2,  3,  4],
        [ 5,  6,  2,  8],
        [ 9,  2, 11,  2],
        [13, 14,  3,  0]]),
 8)

In [28]:
probabilities = [0.7, 0.1, 0.5, 0.8]

prob = np.array(probabilities)
index = prob.argsort()

print(index)

action_dict = {
    0: 'MOVE_RIGHT',
    1: 'MOVE_LEFT',
    2: 'MOVE_UP',
    3: 'MOVE_DOWN'
}

[action_dict[x] for x in index]

[1 2 0 3]


['MOVE_LEFT', 'MOVE_UP', 'MOVE_RIGHT', 'MOVE_DOWN']

In [29]:
from datetime import datetime
import numpy as np

rand_state = np.random.RandomState(datetime.now().microsecond)
seeds = rand_state.randint(np.iinfo(np.int32).max, size=10)
seeds

array([ 927789933,  124277924, 1109188517, 1750150588, 2057306815,
       1147665619,  951327760, 1729951427, 1470617838, 1513745087])

In [5]:
import numpy as np
from src.constants import constants as cts

def bigger_prob(dictionary):
    keys = np.array(list(dictionary.keys()))
    values = np.array(list(dictionary.values()))

    keys_positive = keys[values > 0]

    return keys_positive.max()

d = cts.PROB_NUMBER_NEW_TILES

print(d)
print(bigger_prob(d))

{1: 1, 2: 0}
1


In [56]:
import numpy as np

s = np.array([[1,2,1,0],[1,2,0,0],[3,0,0,0],[3,0,0,1]])

def p(state):
    return np.power(2, state)

def pp(state):
    return np.left_shift(np.ones_like(state), s)

def ppp(state):
    return (2 << state - 1)

print(s)
print(p(s))
print(pp(s))
print(ppp(s))

[[1 2 1 0]
 [1 2 0 0]
 [3 0 0 0]
 [3 0 0 1]]
[[2 4 2 1]
 [2 4 1 1]
 [8 1 1 1]
 [8 1 1 2]]
[[2 4 2 1]
 [2 4 1 1]
 [8 1 1 1]
 [8 1 1 2]]
[[2 4 2 0]
 [2 4 0 0]
 [8 0 0 0]
 [8 0 0 2]]


In [68]:
%%timeit
p(s)

1.01 µs ± 17.5 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [66]:
%%timeit
pp(s)

3.55 µs ± 71.4 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [61]:
%%timeit
ppp(s)

1.7 µs ± 19.3 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
